In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

In [19]:
data = pd.read_csv('SET 50 Historical Data (1).csv')
data.tail(10)


,Date,Price,Open,High,Low,Vol.,Change %
10,04/17/2025,729.31,728.35,733.03,725.50,1.39B,-0.14%
11,04/16/2025,730.31,725.46,733.41,717.92,1.73B,0.45%
12,04/11/2025,727.04,726.12,732.49,718.90,1.63B,-0.38%
13,04/10/2025,729.82,730.53,739.55,720.88,2.13B,4.85%
14,04/09/2025,696.06,682.30,701.24,676.75,2.43B,1.81%
15,04/08/2025,683.69,680.16,687.41,667.69,2.89B,-4.40%
16,04/04/2025,715.17,736.56,737.09,713.61,2.12B,-3.57%
17,04/03/2025,741.67,741.54,744.70,737.49,1.72B,-1.30%
18,04/02/2025,751.43,747.52,754.16,746.25,1.19B,0.51%
19,04/01/2025,747.60,745.48,751.61,745.07,871.77M,1.06%


In [10]:
# secs = [
#     "ADVANC.BK", "AOT.BK", "AWC.BK", "BANPU.BK", "BBL.BK", "BDMS.BK", "BEM.BK", "BGRIM.BK", "BH.BK", "BJC.BK",
#     "BTS.BK", "CBG.BK", "CCET.BK", "COM7.BK", "CPALL.BK", "CPF.BK", "CPN.BK", "CRC.BK", "DELTA.BK", "EGCO.BK",
#     "GLOBAL.BK", "GPSC.BK", "GULF.BK", "HMPRO.BK", "ITC.BK", "IVL.BK", "KBANK.BK", "KTB.BK", "KTC.BK", "LH.BK",
#     "MINT.BK", "MTC.BK", "OR.BK", "OSP.BK", "PTT.BK", "PTTEP.BK", "PTTGC.BK", "RATCH.BK", "SAWAD.BK", "SCB.BK",
#     "SCC.BK", "SCGP.BK", "TISCO.BK", 'TLI.BK', 'TOP.BK', 'TRUE.BK', 'TTB.BK', 'TU.BK', 'VGI.BK', 'WHA.BK'
# ]

secs = [
    "CCET.BK", "CPF.BK", "CRC.BK", "GPSC.BK", "GULF.BK", "HMPRO.BK", "ITC.BK", "IVL.BK", "KBANK.BK", "KTB.BK", "KTC.BK", "LH.BK",
    "MINT.BK", "MTC.BK", "OSP.BK", "PTT.BK", "PTTEP.BK",
    "SCC.BK", "SCGP.BK", "TISCO.BK", 'TLI.BK', 'TOP.BK', 'TRUE.BK', 'TTB.BK', 'TU.BK', 'VGI.BK', 'WHA.BK'
]

In [11]:
df = pd.DataFrame()

for i, inp in enumerate(secs):
    selected_df = yf.download(inp, 
                              start='2019-01-01', 
                              end='2025-05-02', 
                              interval='1d', 
                              rounding=2, 
                              progress=False)
    
    # Flat the multiindex
    if isinstance(selected_df.columns, pd.MultiIndex):
        selected_df.columns = selected_df.columns.get_level_values(0)
    
    # Remove ticker
    if 'Price' in selected_df.columns:
        selected_df = selected_df[selected_df['Price'] != 'Ticker']
    
    # Continuously Compound Rate
    selected_df['Change'] = np.log(selected_df['Close'] / selected_df['Close'].shift(1))
    
    # Change date to column by resetting te index
    selected_df.reset_index(inplace=True)
    
    # initial date in df for merging
    if i == 0:
        df['Date'] = selected_df['Date']
    
    mer_df = selected_df[['Date', 
                          'Close', 
                          ]].copy()
    mer_df = mer_df.rename(columns={'Close': inp})
    
    df = pd.merge(df, mer_df, on='Date', how='outer')

# First row's change is NaN
# df = df.iloc[1:]

print(df.head())

# Save to Excel
with pd.ExcelWriter('data2.xlsx', engine='openpyxl', mode='w') as writer:
    df.to_excel(writer, sheet_name='Combined', index=False)


1 Failed download:
['CCET.BK']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')

1 Failed download:
['CPF.BK']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')

1 Failed download:
['CRC.BK']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')

1 Failed download:
['GPSC.BK']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')

1 Failed download:
['GULF.BK']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')

1 Failed download:
['HMPRO.BK']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')

1 Failed download:
['ITC.BK']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')

1 Failed download:
['IVL.BK']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')

1 Failed download:
['KBANK.BK']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')

1 Failed download:
['KTB.BK']: YFRateLimitError('Too Man

Empty DataFrame
Columns: [Date, CCET.BK, CPF.BK, CRC.BK, GPSC.BK, GULF.BK, HMPRO.BK, ITC.BK, IVL.BK, KBANK.BK, KTB.BK, KTC.BK, LH.BK, MINT.BK, MTC.BK, OSP.BK, PTT.BK, PTTEP.BK, SCC.BK, SCGP.BK, TISCO.BK, TLI.BK, TOP.BK, TRUE.BK, TTB.BK, TU.BK, VGI.BK, WHA.BK]
Index: []

[0 rows x 28 columns]


In [21]:
set_data = pd.read_excel('Data.xlsx')

drop_data = set_data.dropna(axis=1)
drop_data.tail(5)

merged_df = data.join(drop_data, how='inner')

merged_df.tail(10)

ValueError: columns overlap but no suffix specified: Index(['Date'], dtype='object')

KeyError: ('Date', 'Price')